In [1]:
from __future__ import absolute_import
from matplotlib import pyplot as plt

import os
import tensorflow as tf
import numpy as np
import random
import math
from preprocess import get_data
from sklearn import preprocessing
from main import *

2023-05-02 00:10:47.138857: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Import and reshape
X_train, X_test, X_val, Y_train, Y_test, Y_val = get_data(.3, "./../processed_data/")

char_encoder = preprocessing.LabelEncoder().fit(np.concatenate((Y_train.reshape(-1), Y_test.reshape(-1), Y_val.reshape(-1))))
orig_shapes = [Y_train.shape[0], Y_test.shape[0], Y_val.shape[0]]
Y_train = char_encoder.transform(Y_train.reshape(-1)).reshape((orig_shapes[0], 4))
Y_test = char_encoder.transform(Y_test.reshape(-1)).reshape((orig_shapes[1], 4))
Y_val = char_encoder.transform(Y_val.reshape(-1)).reshape((orig_shapes[2], 4))

#Convert input to properly shaped and typed tensors
X_train = tf.convert_to_tensor(np.asarray(np.reshape(X_train, (-1, *X_train.shape[-2:]))).astype(np.float32))
X_val   = tf.convert_to_tensor(np.asarray(np.reshape(X_val  , (-1, *X_val.shape[-2:]))).astype(np.float32))
Y_train = tf.convert_to_tensor(np.asarray(np.reshape(Y_train, (-1))))
Y_val   = tf.convert_to_tensor(np.asarray(np.reshape(Y_val  , (-1))))

#Don't reshape test, because we need to preserve CAPTCHAs but convert to tensor
X_test  = tf.convert_to_tensor(np.asarray(X_test).astype(np.float32))
Y_test  = tf.convert_to_tensor(np.asarray(Y_test))

#Expand Dims
X_train = tf.expand_dims(X_train, axis=-1)
X_val   = tf.expand_dims(X_val  , axis=-1)

#One hot encode
Y_val   = tf.one_hot(Y_val  , depth=31, axis=-1)
Y_train = tf.one_hot(Y_train, depth=31, axis=-1)


model = Model()
print(model(X_train).shape)
model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(.0001),
                metrics= [
                tf.keras.metrics.CategoricalCrossentropy()
                ])
model.fit(
    X_train,
    Y_train,
    epochs=3,
    batch_size=256,
    validation_data=(X_val, Y_val)
)

model.save('./../models/segmented_2')

2023-05-02 00:11:13.459806: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(27872, 31)
Epoch 1/3
109/109 [==============================] - 29s 248ms/step - loss: 1.9290 - categorical_crossentropy: 1.9290 - val_loss: 1.8051 - val_categorical_crossentropy: 1.8051
Epoch 2/3
109/109 [==============================] - 33s 303ms/step - loss: 0.3839 - categorical_crossentropy: 0.3839 - val_loss: 0.5919 - val_categorical_crossentropy: 0.5919
Epoch 3/3
109/109 [==============================] - 30s 279ms/step - loss: 0.1633 - categorical_crossentropy: 0.1633 - val_loss: 0.2853 - val_categorical_crossentropy: 0.2853


INFO:tensorflow:Assets written to: ./../models/segmented/assets


INFO:tensorflow:Assets written to: ./../models/segmented/assets


In [15]:
def run_tests(X_test, Y_test):
    X_test = np.reshape(X_test, (-1, *X_test.shape[-2:]))
    Y_test = np.reshape(Y_test, (-1))
    
    probs = model.predict(X_test, verbose=0)
    diff = np.argmax(probs, axis=1) - Y_test
    accuracy = np.where(diff == 0, 1, 0)

    group_acc = np.array_split(diff, diff.shape[0]//4)
    group_acc = np.apply_along_axis((lambda x: 1 if np.all(x == 0) else 0), axis=1, arr=group_acc)

    return np.mean(accuracy), np.mean(group_acc)
    

output = model.predict(X_test[7], verbose=0)

print("Secret CAPTCHA:")
print(char_encoder.inverse_transform(Y_test[7]))

print("Model guess:")
print(char_encoder.inverse_transform(np.argmax(output, axis=1)))


indiv_acc, group_acc = run_tests(X_test, Y_test)
print(f"Individual accuracy: {round(100*indiv_acc, 2)}% \nCaptcha accuracy: {round(100*group_acc, 2)}%")


Secret CAPTCHA:
['7' 'X' 'R' 'M']
Model guess:
['7' 'X' 'R' 'M']
Individual accuracy: 92.53% 
Captcha accuracy: 73.07%
